<a href="https://colab.research.google.com/github/rahulprasanth487/LLM-Performance-Evaluation/blob/main/Gemma_4bit_version_T4_GPU_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install hf_transfer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!nvidia-smi

Sun Mar 24 06:29:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from IPython.display import Markdown, display
display(Markdown("## Gemma - 2B"))

## Gemma - 2B

In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os


os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [5]:
torch.set_default_device('cuda')

In [6]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", # 'google/gemma-2b-it
                                             quantization_config=quantization_config,
                                             low_cpu_mem_usage=True,
                                             torch_dtype="auto",
                                             device_map="auto"
                                             )

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")



config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [7]:
tokenizer.vocab_size

256000

In [8]:
chat = [
    { "role": "user", "content": "Solve 1+2" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<bos><start_of_turn>user\nSolve 1+2<end_of_turn>\n<start_of_turn>model\n'

In [9]:
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

outputs = model.generate(input_ids=inputs.to(model.device),
                         max_new_tokens=512)


In [10]:
text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
# text = text.replace(prompt, '', 1)

In [11]:
display(Markdown(text))

user
Solve 1+2
model
Sure, the answer is 3.

1 + 2 = 3

In [12]:

 chat = [
    { "role": "user", "content": "Tell about yourself" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<bos><start_of_turn>user\nTell about yourself<end_of_turn>\n<start_of_turn>model\n'

In [13]:
prompt

'<bos><start_of_turn>user\nTell about yourself<end_of_turn>\n<start_of_turn>model\n'

### Prompt Format
```
<start_of_turn>user
What is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>
<start_of_turn>model



```

In [14]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [36]:
import psutil
import resource
import time

def generate(input_text, system_prompt="", max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        {"role": "user", "content": system_prompt + '\n\n' + input_text},
    ]

    prompt = tokenizer.apply_chat_template(messages,
                                           tokenize=False,
                                           add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

    # Measure resource utilization and execution time
    cpu_percent_start = psutil.cpu_percent()
    memory_usage_start = psutil.virtual_memory().used / (1024 ** 3)  # Memory usage in GB
    usage_start = resource.getrusage(resource.RUSAGE_SELF)

    # Record the start time
    start_time = time.time()

    # Generate text
    outputs = model.generate(input_ids=inputs.to(model.device),
                             max_new_tokens=max_length,
                             do_sample=True,
                             temperature=0.1,
                             top_k=50,
                             )

    # Record the end time
    end_time = time.time()

    # Measure resource utilization and execution time after generation
    cpu_percent_end = psutil.cpu_percent()
    memory_usage_end = psutil.virtual_memory().used / (1024 ** 3)  # Memory usage in GB
    usage_end = resource.getrusage(resource.RUSAGE_SELF)

    # Calculate the time taken to generate the first token
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)


    text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = text.replace('user\n'+system_prompt+ '\n\n' + input_text + '\nmodel', '', 1)
    wrapped_text = wrap_text(text)
    display(Markdown(wrapped_text))

    # Print resource utilization and execution time
    print("Wall time:", usage_end.ru_utime - usage_start.ru_utime, "seconds")
    print("CPU Usage:", abs(cpu_percent_end - cpu_percent_start), "%")
    print("Memory Usage:", abs(memory_usage_end - memory_usage_start), "GB")


## Instruction Answering

In [37]:
%%time
generate('What is the fastest car in the world and mention its specifications?',
         system_prompt="You are Gemma, a large language model trained by Google.",
         max_length=256)


Sure, here is the fastest car in the world:

**The Bugatti Chiron** is the fastest car in the world, reaching a speed of up to 285
miles per hour (458 kilometers per hour).

**Specifications:**

* **Top speed:** 285 miles per hour (458 kilometers per hour)
* **Engine:** 8-liter W16 engine with 16 cylinders
* **Power:** 1,500 horsepower
* **Torque:** 1,250 pound-feet (1,630 Newton-meters)
* **Acceleration:** 0 to 60 miles per hour (97 kilometers per hour) in 2.4 seconds
* **Weight:** 4,872 pounds (2,210 kilograms)
* **Length:** 20 feet (6.1 meters)
* **Width:** 7 feet (2.1 meters)
* **Height:** 3.6 feet (1.1 meters)

Wall time: 22.701693999999975 seconds
CPU Usage: 12.5 %
Memory Usage: 0.0034942626953125 GB
CPU times: user 22.7 s, sys: 191 ms, total: 22.9 s
Wall time: 23 s


In [38]:
%%time
generate('What is the difference between a Llama, Gemini and Falcon?',
         system_prompt="Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**Llama:**
- Llama is a herbivore mammal species in the Camelidae family.
- Llama is known for its thick fur, docile temperament, and ability to store large amounts
of water in its stomach.

**Gemini:**
- Gemini is a zodiac sign represented by the twins.
- Gemini is associated with communication, adaptability, and curiosity.

**Falcon:**
- Falcon is a genus of birds of prey in the Falconidae family.
- Falcon is known for its sharp eyesight, speed, and agility in flight.

**Difference:**

- Llama is a mammal, while Gemini is a zodiac sign and Falcon is a bird species.
- Llama has thick fur and stores water in its stomach, while Gemini is associated with
communication and adaptability.
- Falcon is a bird of prey, while Llama is an herbivore.

**Therefore, the difference between a Llama, Gemini and Falcon is:**

- Species classification: Llama is a mammal, Gemini is a zodiac sign, and Falcon is a bird
species.
- Physical characteristics: Llama has thick fur and stores water in its stomach, while
Falcon is a bird of prey.
- Associated traits: Gemini is associated with communication and adaptability, while
Falcon is known for its sharp eyesight and agility in flight.

Wall time: 35.599672 seconds
CPU Usage: 80.2 %
Memory Usage: 0.027385711669921875 GB
CPU times: user 35.6 s, sys: 220 ms, total: 35.8 s
Wall time: 39.4 s


In [40]:
%%time
generate('Write a short email to your friend about his health concern',
         system_prompt="You are Gemma, a large language model trained by Google. Write in personel email format",
         max_length=512)


Subject: Checking in on you, [Friend's name]

Hi [Friend's name],

I was hoping to see how you're doing. I know you've been going through a lot lately, and I
was concerned about your recent health issues. I wanted to see if you're doing alright and
if there's anything I can do to help.

I've been thinking about you and I'm sending you my best wishes. I'm hoping that you're
feeling better soon. If there's anything I can do, please let me know. I'm always here to
lend an ear, or just chat if you need it.

Looking forward to hearing from you soon,

Gemma

Wall time: 16.454542000000004 seconds
CPU Usage: 55.4 %
Memory Usage: 0.005764007568359375 GB
CPU times: user 16.5 s, sys: 147 ms, total: 16.6 s
Wall time: 18 s


In [41]:
%%time
generate("""
Anjali – Hi, Raj. How was your weekend?
Raj – Hey, Anjali. My weekend was great. I watched a great movie.

Anjali – Oh really? What was the name of the movie you watched?

Raj – I watched Avengers Endgame. It is the last movie of the Avengers.

Anjali – Oh, I have watched Avengers Endgame too. I loved the movie.

Raj – Really? Who is your favourite Avenger?

Anjali – I can’t name one! Iron Man, Thor, Captain America, Captain Marvel, Scarlet Witch and Black Widow, to name a few.

Raj – Wow, you have some of the strongest Avengers there! I have the same choice except that I loved Spider Man too.

Anjali – My sister took me to see the movie as soon as it was released. Both me and my sister have been great fans of Avengers since childhood.

Raj – Oh wow! I am myself a big fan of Avengers and have watched all the movies. I too wanted to go to the theatre and watch the movie, but I was out of station for a family function.

Anjali – Oh I see. The movie stood up to all the expectations that the audience had after watching the trailer. In fact, I would say the movie surpassed expectations.

Raj – Very true. There was no better way to finish the Avengers, I believe. The movie just took me through a rollercoaster of emotions.

Anjali – True! Just when I was feeling happy that the Avengers got rid of Thanos for good, the next moment I was bawling my eyes out seeing Iron Man had sacrificed himself to save the world and everyone else.

Raj – We can’t ever see Black Widow, Iron Man and Captain America ever in any Marvel movies.

Anjali – Yes, very sad. Anyway it was nice talking to you. See you tomorrow in school. Bye.

Raj – Same here. Bye.""",
         system_prompt="Summarize the conversation",
         max_length=512)


Sure, here is a summary of the conversation:

Anjali and Raj are having a conversation about the movie Avengers Endgame. They both have
seen the movie and enjoyed it. Anjali has a lot of favorite Avengers, including Iron Man,
Thor, Captain America, Captain Marvel, Scarlet Witch, and Black Widow. Raj has a similar
list of favorite Avengers, but he also loves Spider-Man. They talk about their shared love
of the Avengers and their childhood fandom. They also discuss the emotional impact of the
movie, including the death of Iron Man and Black Widow. They end the conversation by
saying goodbye and looking forward to seeing each other in school the next day.

Wall time: 16.075192000000015 seconds
CPU Usage: 49.5 %
Memory Usage: 0.009136199951171875 GB
CPU times: user 16.1 s, sys: 780 ms, total: 16.9 s
Wall time: 16.9 s


In [43]:
%%time
generate("""
Friend 1: Hey, how are you doing?
Friend 2: Not so great, to be honest. I've been feeling really down lately.
Friend 1: Oh no, what's been going on?
Friend 2: It's just... everything feels so overwhelming. I'm struggling with work, and I don't even know where to start.
Friend 1: I'm really sorry to hear that. Is there anything I can do to help?
Friend 2: I appreciate it, but I don't even know what would help at this point. I just feel so lost.
Friend 1: I understand. Sometimes it's hard to see a way forward when everything seems dark. Just know that I'm here for you, no matter what.
Friend 2: Thanks, it means a lot to me. I'm just really tired of feeling like this all the time.
Friend 1: I wish I could take away your pain, but I know it doesn't work that way. Just know that I care about you, and I'll always be here to listen.
Friend 2: Thank you, that means more to me than you'll ever know.
""",
         system_prompt="Classifiy the emotion state in simple terms of these guys using the conversation",
         max_length=512)

user
Classifiy the emotion state in simple terms of these guys using the conversation


Friend 1: Hey, how are you doing?
Friend 2: Not so great, to be honest. I've been feeling really down lately.
Friend 1: Oh no, what's been going on?
Friend 2: It's just... everything feels so overwhelming. I'm struggling with work, and I
don't even know where to start.
Friend 1: I'm really sorry to hear that. Is there anything I can do to help?
Friend 2: I appreciate it, but I don't even know what would help at this point. I just
feel so lost.
Friend 1: I understand. Sometimes it's hard to see a way forward when everything seems
dark. Just know that I'm here for you, no matter what.
Friend 2: Thanks, it means a lot to me. I'm just really tired of feeling like this all the
time.
Friend 1: I wish I could take away your pain, but I know it doesn't work that way. Just
know that I care about you, and I'll always be here to listen.
Friend 2: Thank you, that means more to me than you'll ever know.
model
**Friend 1:**
- **Happy, supportive, and empathetic:**
- Shows concern for Friend 2's well-being and offers comfort and understanding.
- Uses empathetic language like "oh no" and "I understand."

**Friend 2:**
- **Sad, overwhelmed, and lost:**
- Expresses feelings of being down and struggling with work.
- Uses words like "not so great," "overwhelmed," and "lost."

**Overall:**
The conversation indicates a conversation between two friends, where Friend 1 is showing
empathy and support to Friend 2 who is going through a difficult time. The emotion state
of Friend 1 is characterized by concern, empathy, and willingness to listen. The emotion
state of Friend 2 is characterized by sadness, overwhelm, and feeling lost.

Wall time: 20.134476999999947 seconds
CPU Usage: 51.00000000000001 %
Memory Usage: 0.005016326904296875 GB
CPU times: user 20.1 s, sys: 539 ms, total: 20.7 s
Wall time: 21 s
